# Lab 5 - Parcel Feature Extraction

Next, we will illustrate the construction of features related to our main task: finding the relationship between property development and water quality over time.  In a previous lab, you identified lakes for which we have complete information for the years from 2004 to 2015.  In this lab, we will

[Original Data and variable information](https://gisdata.mn.gov/organization/us-mn-state-metrogis?q=Metro+Regional+Parcel+Dataset&sort=score+desc%2C+metadata_modified+desc)

## Problem 1 - Feature construction

**Overview.** Remember that our target output file will have one row per year-lake combination.  To attach property information, we will need to group and aggregate the parcel data to create features for each lake-year combination.  When grouping the data, be sure to maintain the variables needed to join to the water quality data, namely the lake ID and year.  Since we are looking at tracking property development/change over time, we will want to generate features tracking

* Number of properties close to each lake,
* Summaries of the value of properties close to each lake,
* Aggregations on the size and type of the properties, and
* Other features that might impact water quality.
    
#### Task 1. Understanding parcel variables

Before we can construct features, we need to make sure we understand the parcel data.  The metro parcel data is provided by the State of Minnesota and the meta data can be found online.  For example, searching for *metro parcel 2014* lead to [this site](https://geo.btaa.org/catalog/304cf3d8-a53b-4ea9-b02a-f550bd68e320).  Clicking on the *Meta data* button in the top left, brought up more information.  Clicking *Download* opened in this meta data [in a separate page](https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_parcels_2014/metadata/metadata.html)

Look through the **Section 4: Attributes** and identify variables that might impact the water quality of near-by lakes.

> <font color="orange"> Your thoughts here </font>
> 1) Parcel Area / Lost Size. Larger parcels may contribute more runoff
> 2) Land USe / Property Type.
> 3) Building Type / Year Built. DIfferent stormwater controls based on the year
> 4) Assessed Property Value
> 5) Tax Class / Property Use Code
> 6) Ownership Type (public vs. private)
> 7) Parcel Location

### Task 2. Feature Brainstorming

Our objective is to build a feature table with one row per lake-year, using grouped summary statistics. Here are effective strategies for feature construction:

1. **Numerical summaries:** Calculate group-level statistics (mean, median, standard deviation, IQR, etc.) for numeric variables.
2. **Categorical summaries:** For text data, consider:
   - **Success rates:** Compute proportions for binary variables (e.g., percent of homes with basements).
   - **Label cleaning:** Review and standardize unique labels to remove duplicates or inconsistencies.
   - **Broader categories:** Recode variables with many rare categories into a smaller, more meaningful set.
   - **Indicator columns:** Create indicator variables and aggregate them to show presence/absence or proportions (e.g., count of each property use type).

Review the variables you identified earlier and outline a feature construction strategy for each.

> <font color="orange"> Your thoughts here </font>
> I think the most important part of building this feature table is ensuring that each lake‑year row captures both numeric summaries and categorical indicators in a consistent way. By carefully cleaning labels and aggregating proportions, we can make the features more interpretable and useful for downstream analysis. I will possibly start by making a numeric feature table and a categorical feature table then combinng.

### Task 4. Initial querying with filter and select

First, you should build a query that filters the parcel data to 
1. only include parcels within 1600 feet of the lakes we are studying, and 
2. only for the lakes with complete information.  

You should also select only the columns you will need for feature construction and joining to the water quality data.

In [62]:
import polars as pl

parcels = (
    pl.scan_parquet("data/MinneMUDAC_raw_files/parcel_combined_by_year.parquet")
    .filter(
        pl.col("distance_category").is_in([
            "within_500m",
            "Between_501-1600m"
        ])
    )
)

## Problem 2.  Numerical Summaries

Two important categories of property data involve the size (e.g., finished square footage) and value (e.g., accessed value and/or taxes paid).

**Tasks.** 

1. Identify 2-3 variables for each of these categories.
2. Write a query that computes the summary statistics for each of these variables for each lake-year.  
3. Write this summary table out to a CSV file named `parcel_numerical_summaries.csv`.  Again, you should partition by lake ID and year.

In [66]:
numerical_summaries = (
    parcels
    .with_columns([
        pl.col("ACRES_POLY").cast(pl.Float64),
        pl.col("FIN_SQ_FT").cast(pl.Float64),
        pl.col("EMV_TOTAL").cast(pl.Float64),
        pl.col("TOTAL_TAX").cast(pl.Float64),
    ])
    .group_by(["Monit_MAP_CODE1", "Year"])
    .agg([
        pl.col("ACRES_POLY").mean().alias("mean_acres"),
        pl.col("ACRES_POLY").median().alias("median_acres"),

        pl.col("FIN_SQ_FT").mean().alias("mean_sqft"),
        pl.col("FIN_SQ_FT").median().alias("median_sqft"),

        pl.col("EMV_TOTAL").mean().alias("mean_EMV_TOTAL"),
        pl.col("EMV_TOTAL").median().alias("median_EMV_TOTAL"),

        pl.col("TOTAL_TAX").mean().alias("mean_total_tax"),
        pl.col("TOTAL_TAX").median().alias("median_total_tax"),
    ])
)
(numerical_summaries.collect().write_csv("./data/MinneMUDAC_raw_files/parcel_numerical_summaries.csv"))

## Problem 3.  Simple categorical summaries.

In this part, you will create summary statistics for some of the simpler categorical variables.

**Binary variables.** There are two examples of binary variables, listed below.  You will need to compute the percent of `Yes` for each.

* GARAGE: Garage Y/N
* BASEMENT: Basement Y/N

**Other categorical variables.** There are a number of other categorical variables.  You need to select one of these variables, inspect/clean your variable as needed, create indicator variables for each resulting label, and compute summary statistics for each label.

* HOMESTEAD: Homestead Status
* TAX_EXEMPT: Tax Exempt Status 
* DWELL_TYPE: Dwelling Type 
* HOME_STYLE: Home Style
* HEATING: Heating type
* COOLING: Cooling type

**Tasks.**
Create a query that

1. Select one binary and two other categorical variables for feature construction,
2. Reads in the parcel data and selects the relevant columns (be sure to keep the lake ID and year),
3. Inspect unique labels and recode/clean as needed,
4. Create a literal column of ones, and
5. Pivot to get the counts of each label per lake-year (do this once per category).

Write this summary table out to a csv file named `parcel_categorical_summaries.csv`.  Again, you should partition by lake ID and year.

In [ ]:
categorical_summaries = (
    parcels
    .with_columns([
        pl.col("GARAGE").cast(pl.Utf8).str.to_uppercase().fill_null(pl.lit("MISSING")).alias("GARAGE_CLEAN"),
        pl.col("HOMESTEAD").cast(pl.Utf8).str.to_uppercase().fill_null(pl.lit("MISSING")).alias("HOMESTEAD_CLEAN"),
        pl.col("DWELL_TYPE").cast(pl.Utf8).str.to_uppercase().fill_null(pl.lit("MISSING")).alias("DWELL_TYPE_CLEAN"),
    ])
    .group_by(["Monit_MAP_CODE1", "Year"])
    .agg([
        (pl.col("GARAGE_CLEAN") == pl.lit("Y")).sum().alias("count_GARAGE_Y"),
        pl.col("Monit_MAP_CODE1").count().alias("total_parcels"),
        (pl.col("HOMESTEAD_CLEAN") == pl.lit("Y")).sum().alias("count_HOMESTEAD_Y"),
        (pl.col("DWELL_TYPE_CLEAN") == pl.lit("CONDO")).sum().alias("count_DWELL_TYPE_CONDO"),
        (pl.col("DWELL_TYPE_CLEAN") == pl.lit("SINGLE FAMILY")).sum().alias("count_DWELL_TYPE_SINGLE_FAMILY"),
        (pl.col("DWELL_TYPE_CLEAN") == pl.lit("MISSING")).sum().alias("count_DWELL_TYPE_MISSING"),
    ])
    .with_columns([
        (pl.col("count_GARAGE_Y") / pl.col("total_parcels")).alias("pct_GARAGE_Y"),
        (pl.col("count_HOMESTEAD_Y") / pl.col("total_parcels")).alias("pct_HOMESTEAD_Y"),
        (pl.col("count_DWELL_TYPE_CONDO") / pl.col("total_parcels")).alias("pct_DWELL_TYPE_CONDO"),
        (pl.col("count_DWELL_TYPE_SINGLE_FAMILY") / pl.col("total_parcels")).alias("pct_DWELL_TYPE_SINGLE_FAMILY"),
        (pl.col("count_DWELL_TYPE_MISSING") / pl.col("total_parcels")).alias("pct_DWELL_TYPE_MISSING"),
    ])
)

categorical_summaries.collect().write_csv("./data/MinneMUDAC_raw_files/parcel_categorical_summaries.csv")

## Problem 4.  Join all the summaries.

Finally, you need to join all the summaries created above, along with the water quality summaries created in a previous lab, into one overall summary file.  Write the resulting table to a CSV file named `water_quality_and_parcel_summaries_2004_to_2015.csv`.

In [72]:
(numerical_summaries := pl.read_csv("./data/MinneMUDAC_raw_files/parcel_numerical_summaries.csv"))

Monit_MAP_CODE1,Year,mean_acres,median_acres,mean_sqft,median_sqft,mean_EMV_TOTAL,median_EMV_TOTAL,mean_total_tax,median_total_tax
str,i64,f64,f64,f64,f64,f64,f64,f64,f64
"""02001300-01""",2011,0.901463,0.29,1481.013118,1441.0,204211.200807,193500.0,3325.578204,2974.0
"""27008200-01""",2005,0.0,0.0,0.0,0.0,256200.350877,238200.0,2767.852632,2527.0
"""82004200-02""",2014,9.32125,6.03,1074.78125,1125.0,284846.875,336950.0,0.0,0.0
"""19002200-01""",2013,0.583347,0.37,2257.133065,2198.0,226803.528226,223900.0,2923.899194,2862.0
"""82000500-01""",2012,4.802143,4.995,1949.357143,2081.5,407621.428571,374000.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…
"""19009500-01""",2010,0.207006,0.14,1772.793787,1596.0,178039.391937,161400.0,2196.208196,1960.0
"""70007600-01""",2010,5.784,1.525,1187.26,1280.5,416384.0,287250.0,0.0,0.0
"""82002602-01""",2007,0.0,0.0,0.0,0.0,423288.888889,410150.0,0.0,0.0


In [71]:
(categorical_summaries := pl.read_csv("./data/MinneMUDAC_raw_files/parcel_categorical_summaries.csv"))

Monit_MAP_CODE1,Year,count_GARAGE_Y,total_parcels,count_HOMESTEAD_Y,count_DWELL_TYPE_CONDO,count_DWELL_TYPE_SINGLE_FAMILY,count_DWELL_TYPE_MISSING,pct_GARAGE_Y,pct_HOMESTEAD_Y,pct_DWELL_TYPE_CONDO,pct_DWELL_TYPE_SINGLE_FAMILY,pct_DWELL_TYPE_MISSING
str,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64
"""82005400-01""",2004,0,298,298,0,0,298,0.0,1.0,0.0,0.0,1.0
"""27017800-01""",2014,69,108,95,0,0,108,0.638889,0.87963,0.0,0.0,1.0
"""27011800-02""",2004,0,1223,1081,0,0,1223,0.0,0.883892,0.0,0.0,1.0
"""82010800-01""",2011,98,146,78,0,0,46,0.671233,0.534247,0.0,0.0,0.315068
"""82010100-01""",2004,0,290,290,0,0,290,0.0,1.0,0.0,0.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…
"""10004200-01""",2007,6,33,13,0,0,33,0.181818,0.393939,0.0,0.0,1.0
"""82001400-01""",2014,110,134,100,0,0,22,0.820896,0.746269,0.0,0.0,0.164179
"""10008800-01""",2009,23,71,30,0,0,71,0.323944,0.422535,0.0,0.0,1.0


In [74]:
(water_quality := pl.read_csv("./data/MinneMUDAC_raw_files/water_quality_summaries_complete.csv"))

DNR_ID_Site_Number,LAKE_NAME,Year,Phosphorus_mean,Secchi_mean
str,str,i64,f64,f64
"""19002200-01""","""Long Lake""",2010,76.305085,1.804576
"""19034800-01""","""Valley Lake""",2009,54.142857,2.285714
"""82011301-01""","""Goose Lake""",2011,93.6,0.909
"""19002200-01""","""Long Lake""",2004,87.5625,1.364328
"""82013700-01""","""Fish Lake""",2008,58.5,1.502722
…,…,…,…,…
"""10000200-01""","""Riley Lake""",2005,51.642857,2.05
"""82011602-01""","""Armstrong Lake""",2006,89.0,1.044857
"""19034800-01""","""Valley Lake""",2006,88.142857,1.1


In [73]:
numerical_categorical = numerical_summaries.join(
    categorical_summaries,
    on=["Monit_MAP_CODE1", "Year"],
    how="inner"
)

In [83]:
numerical_categorical_water_quality = numerical_categorical.join(
    water_quality,
    left_on=["Monit_MAP_CODE1", "Year"],
    right_on=["DNR_ID_Site_Number", "Year"],
    how="inner"
)
numerical_categorical_water_quality = numerical_categorical_water_quality.drop_nulls()

In [85]:
numerical_categorical_water_quality.write_csv("./data/MinneMUDAC_raw_files/numerical_categorical_water_quality.csv")

## Problem 5.  Put it all together

It is often useful to package all of the data constructions steps together in one convenient place.  Your last task is to

1. Gather all of your data construction code below.
    * You don't need to include exploratory code, e.g., exploring join mismatches; only the code necessary to combine, clean, and write your data.
2. Clean/refactor the code.
3. Be sure to display all important intermediate results.

In [80]:
numerical_categorical_water_quality.head()

Monit_MAP_CODE1,Year,mean_acres,median_acres,mean_sqft,median_sqft,mean_EMV_TOTAL,median_EMV_TOTAL,mean_total_tax,median_total_tax,count_GARAGE_Y,total_parcels,count_HOMESTEAD_Y,count_DWELL_TYPE_CONDO,count_DWELL_TYPE_SINGLE_FAMILY,count_DWELL_TYPE_MISSING,pct_GARAGE_Y,pct_HOMESTEAD_Y,pct_DWELL_TYPE_CONDO,pct_DWELL_TYPE_SINGLE_FAMILY,pct_DWELL_TYPE_MISSING,LAKE_NAME,Phosphorus_mean,Secchi_mean,LAKE_NAME_right,Phosphorus_mean_right,Secchi_mean_right
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,str,f64,f64,str,f64,f64
"""19002200-01""",2010,0.578634,0.37,2170.702891,2160.0,236843.469591,235900.0,3204.801595,3000.0,0,1003,913,0,0,75,0.0,0.910269,0.0,0.0,0.074776,"""Long Lake""",76.305085,1.804576,"""Long Lake""",76.305085,1.804576
"""19034800-01""",2009,0.395818,0.27,1859.755427,1826.0,213134.298119,194500.0,2277.976845,2244.0,0,691,586,0,0,20,0.0,0.848046,0.0,0.0,0.028944,"""Valley Lake""",54.142857,2.285714,"""Valley Lake""",54.142857,2.285714
"""82011301-01""",2011,2.607586,1.32,1849.224138,2054.0,466731.034483,426250.0,0.0,0.0,66,116,54,0,0,48,0.568966,0.465517,0.0,0.0,0.413793,"""Goose Lake""",93.6,0.909,"""Goose Lake""",93.6,0.909
"""19002200-01""",2004,0.592285,0.37,2151.954499,2132.0,235813.751264,235600.0,2254.88271,2254.0,0,989,914,0,0,61,0.0,0.924166,0.0,0.0,0.061678,"""Long Lake""",87.5625,1.364328,"""Long Lake""",87.5625,1.364328
"""82013700-01""",2008,9.906316,6.295,274.144737,0.0,523681.578947,501150.0,451359.868421,469350.0,2,76,8,0,0,68,0.026316,0.105263,0.0,0.0,0.894737,"""Fish Lake""",58.5,1.502722,"""Fish Lake""",58.5,1.502722
